In [11]:
import os
import sys

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob

from pandeia.engine.calc_utils import build_default_calc, build_default_source
from pandeia.engine.perform_calculation import perform_calculation

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='../../config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleSkyPyLens
from mejiro.plots import diagnostic_plot, plot, plot_util, overplot
from mejiro.analysis import stats
from mejiro.utils import util
from mejiro.helpers import pyhalo, lenstronomy_sim, pandeia_input

In [12]:
bands = util.hydra_to_dict(config.pipeline)['band']

In [13]:
lens = SampleSkyPyLens()
model = lens.get_array(num_pix=97, side=10.67)

In [14]:
images = []

In [15]:
for band in bands:
    calc = build_default_calc('roman', 'wfi', 'imaging')
    calc['scene'][0]['spectrum']['normalization']['norm_wave'] = pandeia_input._get_norm_wave(band.lower())
    calc['scene'][0]['spectrum']['normalization']['norm_fluxunit'] = 'mjy'
    calc['scene'][0]['spectrum']['normalization']['norm_waveunit'] = 'microns'
    calc['scene'][0]['spectrum']['normalization']['type'] = 'at_lambda'

    print(calc['scene'][0]['spectrum'])

    # set scene size settings
    calc['configuration']['max_scene_size'] = 5.

    # set instrument
    calc['configuration']['instrument']['filter'] = band.lower()  # e.g. 'f106'

    # set detector
    calc['configuration']['detector']['ma_table_name'] = 'hlwas_imaging'

    # turn off noise sources
    calc['calculation'] = pandeia_input.get_calculation_dict(init=False)

    # add an extra point source far out to force maximum scene size
    calc['scene'].append(build_default_source(geometry='point', telescope='roman'))
    calc['scene'][1]['position']['x_offset'] = 100
    calc['scene'][1]['position']['y_offset'] = 100

    image, _ = pandeia_input.get_pandeia_image(calc)
    images.append(image)

{'name': 'generic source', 'redshift': 0.0, 'lines': [], 'sed': {'sed_type': 'flat', 'unit': 'fnu', 'z': 0.0}, 'normalization': {'type': 'at_lambda', 'norm_wave': 1.06, 'norm_flux': 0.001, 'norm_fluxunit': 'mjy', 'norm_waveunit': 'microns'}, 'extinction': {'law': 'mw_rv_31', 'unit': 'mag', 'bandpass': 'j', 'value': 0.0}}
Performing Pandeia calculation...
Pandeia calculation complete in 0:00:01
{'name': 'generic source', 'redshift': 0.0, 'lines': [], 'sed': {'sed_type': 'flat', 'unit': 'fnu', 'z': 0.0}, 'normalization': {'type': 'at_lambda', 'norm_wave': 1.293, 'norm_flux': 0.001, 'norm_fluxunit': 'mjy', 'norm_waveunit': 'microns'}, 'extinction': {'law': 'mw_rv_31', 'unit': 'mag', 'bandpass': 'j', 'value': 0.0}}
Performing Pandeia calculation...
Pandeia calculation complete in 0:00:01
{'name': 'generic source', 'redshift': 0.0, 'lines': [], 'sed': {'sed_type': 'flat', 'unit': 'fnu', 'z': 0.0}, 'normalization': {'type': 'at_lambda', 'norm_wave': 1.577, 'norm_flux': 0.001, 'norm_fluxunit'

In [16]:
plot.plot(image, colorbar=True, title=f'{image.shape}')

In [17]:
maxes = [np.max(i) for i in images]
print(maxes)

[4.765629510460322, 4.009278614169467, 3.227044734375456, 1.9691718210052678]


In [19]:
mjy_to_counts, counts_to_mjy = [], []
for i, max in enumerate(maxes):
    mjy_to_counts.append(maxes[i] / 0.001)
    counts_to_mjy.append(0.001 / maxes[i])

print(mjy_to_counts)
print(counts_to_mjy)

[4765.629510460321, 4009.2786141694673, 3227.044734375456, 1969.1718210052677]
[0.00020983586697309335, 0.0002494214286993753, 0.0003098810466888475, 0.000507827701642357]
